In [2]:
%%bash

python scClassifier3.py --sup-data-file /home/wsa/data/mtx_file/ref_dataset.mtx \
                        --sup-label-file /home/wsa/data/mtx_file/ref_dataset_cluster.txt \
                        --unsup-data-file /home/wsa/data/mtx_file/alt_dataset.mtx \ 
                        -lr 0.0001 \
                        -n 100 \
                        -bs 1000 \
                        --aux-loss \
                        -ba \
                        --validation-fold 10 \
                        --cuda \
                        --save-model bench_allglioma_with_tumor_model.pth
                        


1 epoch: avg losses 2107244.2125 0.0000 validation accuracy 0.0138 F1 0.0027(macro) 0.0004(weighted) precision 0.0014 recall 0.1000 mcc 0.0000 elapsed 12.5171 seconds


Traceback (most recent call last):
  File "/home/wsa/code/glioma/scClassifier3v2/scClassifier3.py", line 807, in <module>
    main(args)
  File "/home/wsa/code/glioma/scClassifier3v2/scClassifier3.py", line 520, in main
    epoch_losses_sup, epoch_losses_unsup = run_inference_for_epoch(
  File "/home/wsa/code/glioma/scClassifier3v2/scClassifier3.py", line 348, in run_inference_for_epoch
    new_loss = losses[loss_id].step(xs, ys)
  File "/home/wsa/.conda/envs/wsa/lib/python3.9/site-packages/pyro/infer/svi.py", line 145, in step
    loss = self.loss_and_grads(self.model, self.guide, *args, **kwargs)
  File "/home/wsa/.conda/envs/wsa/lib/python3.9/site-packages/pyro/infer/traceenum_elbo.py", line 451, in loss_and_grads
    for model_trace, guide_trace in self._get_traces(model, guide, args, kwargs):
  File "/home/wsa/.conda/envs/wsa/lib/python3.9/site-packages/pyro/infer/traceenum_elbo.py", line 394, in _get_traces
    yield self._get_trace(model, guide, args, kwargs)
  File "/home/wsa/.

Error while terminating subprocess (pid=58642): 


In [4]:
import torch

In [5]:
%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
from scipy.io import mmread
from scClassifier3 import scClassifier3
from utils.scdata_cached import SingleCellCached, transform_label2class, label2class_encoder, fn_y_scdata

import torch
from torch.utils.data import DataLoader
import pyro.distributions as dist

from sklearn.metrics import accuracy_score, f1_score, normalized_mutual_info_score, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler, Normalizer, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.mixture import BayesianGaussianMixture

import umap
import numpy as np

#import rpy2.robjects.packages as packages
#import rpy2.robjects.lib.ggplot2 as ggplot2
#import rpy2.robjects as ro
#from rpy2.robjects import pandas2ri

#from rpy2.robjects.conversion import localconverter


plt.rcParams["figure.figsize"] = (20, 10)


In [6]:
ModelPath = 'bench_glioma_with_tumor_model.pth'


altDataPath='/home/wsa/data/mtx_file1/alt_dataset.mtx'
altLabelPath=None


refDataPath='/home/wsa/data/mtx_file1/ref_dataset.mtx'
refLabelPath='/home/wsa/data/mtx_file1/ref_dataset_cluster.txt'



In [7]:
# load model
model = torch.load(ModelPath)

In [8]:
# load data
batch_size = 100
use_float64 = False
use_cuda = True

alt_data_cached = SingleCellCached(altDataPath, altLabelPath, model.class2label, 'valid', use_cuda=False, use_float64 = use_float64)
alt_data_loader = DataLoader(alt_data_cached, batch_size = batch_size, shuffle = False)

ref_data_cached = SingleCellCached(refDataPath, refLabelPath, model.class2label, 'valid', use_cuda=False, use_float64 = use_float64)
ref_data_loader = DataLoader(ref_data_cached, batch_size = batch_size, shuffle = False)


In [9]:
# predict conditions
#embeds顺序是先alt再ref
embeds = []

# use the appropriate data loader
for (xs, ys) in alt_data_loader:
    # use classification function to compute all predictions for each batch
    embed = model.latent_embedding(xs)
    embed = embed.cpu().detach().numpy()
    embeds.append(embed)

for (xs, _) in ref_data_loader:
    # use classification function to compute all predictions for each batch
    embed = model.latent_embedding(xs)
    embed = embed.cpu().detach().numpy()
    embeds.append(embed)

embeds = np.concatenate(embeds, 0)


In [10]:
#pos = umap.UMAP(metric='cosine').fit_transform(embeds)
pos = umap.UMAP(metric='cosine', min_dist=0.3).fit_transform(embeds)

In [11]:
pd.DataFrame(pos).to_csv('/home/wsa/data/bench_allglioma_with_tumor_dataset_scp_umap.txt')
pd.DataFrame(embeds).to_csv('/home/wsa/data/bench_allglioma_with_tumor_dataset_scp_embed.txt')


In [12]:
pos

array([[ 8.226536  , -0.9431042 ],
       [ 9.305362  , -0.72358406],
       [10.220177  ,  3.7061162 ],
       ...,
       [ 0.5446433 , 11.32099   ],
       [ 2.8834095 , 15.416737  ],
       [ 4.4273167 , 15.697309  ]], dtype=float32)

In [14]:
pos.shape

(80745, 2)

In [15]:
# predict conditions
preds = []

# use the appropriate data loader
for (xs, ys) in alt_data_loader:
    # use classification function to compute all predictions for each batch
    pred = model.classifier(xs)
    #pred = pred.detach().numpy()
    _, pred = torch.topk(pred, 1)
    pred = model.convert_to_label(pred)
    preds.append(pred)


preds = np.concatenate(preds, 0)

In [17]:
pd.DataFrame(preds).to_csv('/home/wsa/data/mtx_file1/alt_cluster_predict.txt')